# Library Imports

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score,accuracy_score, precision_score, recall_score, f1_score, log_loss
import pandas as pd
import numpy as np
from aif360.metrics import ClassificationMetric
from aif360.datasets import BinaryLabelDataset


# Data Load

In [5]:
df = pd.read_parquet('data/nhanes_data_processed.parquet')
df.head()

,Weight,Body mass index,Systolic,Diastolic,Gender,Age,Diabetes,Glycohemoglobin,Cholesterol,High-density lipoprotein (HDL),...,Basophils,Red blood cells,Hemoglobin,Red blood cell width,Platelet count,Mean volume of platelets,Coronary heart disease,Blood related diabetes,Moderate-work,Vigorous-work
4,92.5,29.10,122.0,82.0,Male,597.0,No,5.5,7.21,1.08,...,5.397605e-79,5.13,14.5,13.1,209.0,10.4,No,No,17.0,Yes
6,78.0,29.39,130.0,78.0,Female,712.0,No,5.8,6.34,2.73,...,5.397605e-79,4.60,13.4,14.3,244.0,8.2,No,Yes,3.0,No
9,111.8,30.94,152.0,98.0,Male,518.0,No,5.5,3.62,1.31,...,5.397605e-79,5.00,15.4,13.7,167.0,9.4,No,Don't know,13.0,Don't know
13,75.5,27.33,142.0,56.0,Male,973.0,No,5.8,4.50,1.04,...,5.397605e-79,5.32,16.6,12.4,160.0,9.0,No,No,9.0,Yes
14,81.6,26.68,106.0,68.0,Female,459.0,No,4.6,5.15,1.49,...,5.397605e-79,4.14,13.3,11.9,255.0,7.7,No,No,13.0,Yes


# Data Processing

In [6]:
def process_and_load_data(data = pd.DataFrame, target = str):
    #The input should only be a Pandas DataFrame 
 
    #This creates split datasets for training, testing, and validation
    #Additionally it prepares the input data sets for model fitting and predicting
    X = data.drop(target, axis = 1)
    y = data[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    X_val =  scaler.transform(X_val)

    return X_train, X_test, X_val, y_train, y_test, y_val

# Main Model

In [ ]:
class main_model():
    def __init__(self, input_dim):
        super(main_model, self).__init__()
        self.dense1 = Dense(32, activation='relu')
        self.dense2 = Dense(16, activation='relu')
        self.output_layer = Dense(1, activation='sigmoid') 

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.output_layer(x)

In [8]:
def main_model(input):
    model = Sequential([
    Dense(units=32, activation='relu', input_shape=(input,)),  
    Dropout(0.3),  
    Dense(units=16, activation='relu'),  
    Dense(1, activation='sigmoid') 
    ])

    return model

In [12]:

adam = keras.optimizers.Adam(learning_rate=0.001)
main_model(41).compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])

/home/dsilva/DSC-180-Capstone/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Adversarial Model 

In [ ]:
def adversarial_model():
    return tfdf.keras.GradientBoostedTreesModel()

# Custom Fit() - Adversarial Debaising

In [3]:
class AdversarialModel(keras.Model):
    def __init__(self, input_dim, sensitive_attr = str,lambda_tradeoff=0.1, metric = str, GBT_retrain = 5, epochs = 100, num_batches = 32):
        super(AdversarialModel, self).__init__()

        # Initialize Attributes
        self.lambda_tradeoff = lambda_tradeoff  # Trade-off parameter for adversarial penalty
        self.sensitive_attr = sensitive_attr
        self.epochs = epochs
        self.GBT_retrain = GBT_retrain
        self.metric = metric
        self.num_batches = num_batches
   
        # Define the main neural network
        self.dense1 = Dense(32, activation='relu', input_dim = input_dim)
        self.dropout1 = Dropout(0.3)  # Added Dropout layer
        self.dense2 = Dense(16, activation='relu')
        self.output_layer = Dense(1, activation='sigmoid')  # Binary classification
    
        
        # Loss function and optimizer
        self.loss_fn = keras.losses.BinaryCrossentropy()
        self.optimizer = keras.optimizers.Adam(learning_rate=0.001)

        # Adversarial model (Gradient Boosted Trees)
        self.adversarial_model = tfdf.keras.GradientBoostedTreesModel()(n_estimators=100, learning_rate=0.1)



    def call(self, inputs):
        """Forward pass"""
        x = self.dense1(inputs)
        x = self.dropout1(x, training = True)
        x = self.dense2(x)
        return self.output_layer(x)

    def fit(self, data):
            
        z = X_train[self.sensitive_attr]
            
        for epoch in range(self.epochs):

            # Epoch Progress Tracking
            print(f"\nEpoch {epoch+1}/{self.epochs}")
            progbar = keras.utils.Progbar(target=self.num_batches)

            for X_train, y_train in data:
                
                if if self.epoch % self.GBT_retrain == 0:





                with tf.GradientTape() as tape:
                    # Train Main Model
                    # Forward pass
                    y_pred = self(X_train, training=True)  

                    # Compute Main Model Loss
                    main_model_loss = self.loss_fn(y_train, y_pred)

                    # Compute gradients
                    trainable_vars = self.trainable_variables
                    gradients = tape.gradient(main_model_loss, trainable_vars)

                    # Update weights
                    self.optimizer.apply_gradients(zip(gradients, trainable_vars))


            # Train Adversarial Model
            # Depending on Metric Input may Require 

            
                zpred = 
                 
      

            
            # Compute Combined Loss
            #combined_loss = main_model_loss + main

            # Compute our own metrics
            self.loss_tracker.update_state(loss)
            self.mae_metric.update_state(y, y_pred)
            return {"loss": self.loss_tracker.result(), "mae": self.mae_metric.result()}

    ### Step 5: Update Metrics and Return Results ###
    self.compiled_metrics.update_state(y, y_pred)
    results = {m.name: m.result() for m in self.metrics}
    results["adversary_loss"] = loss_r  # Log adversary loss
    return results

IndentationError: unexpected indent (3569376808.py, line 43)